In [1]:
import pandas
iolcsvfile = "iolbonos.csv"
iolbonosdf = pandas.read_csv(iolcsvfile, index_col="Simbolo")
#iolbonosdf

In [2]:
fees = (5/1000, 3/1000, 1/1000)
namefee = ("gold", "platinum", "black")
comissions = dict(zip(namefee, fees))

In [3]:
def tcventa(ars, usd, fee="gold"):
    return round((ars * (1 - comissions[fee]))/(usd * (1 + comissions[fee])), 2)

def tccompra(ars, usd, fee="gold"):
    return round((ars * (1 + comissions[fee]))/(usd * (1 - comissions[fee])), 2)

# Vale la pena crear variables locales para cada parte del numerador y denominador?
# revisar los decimales
#FALTA INCORPORAR LOS DERECHOS DE MERCADO
#FALTA INCLUIR VALORES MINIMOS
# Compra no tiene en cuenta el parking

In [4]:
indexlist = (iolbonosdf.index.values.tolist())
# Es realmente necesario el ".values."?
# Hace falta q este entre parentesis todo el coso?

mep = [i for i in indexlist if i.upper().endswith("D")]
mepstrip = [i.rstrip(i[-1]) for i in mep]

ccl = [i for i in indexlist if i.upper().endswith("C")]
cclstrip = [i.rstrip(i[-1]) for i in ccl]

exclude = []

In [5]:
for i in mepstrip:
    if i not in indexlist:
        exclude.append(i + "D")
        
for i in cclstrip:
    if i not in indexlist:
        exclude.append(i + "C")

In [6]:
for i in exclude: 
    if i in mep:
        mep.remove(i)

    if i in ccl:
        ccl.remove(i)

In [7]:
mepstrip = [i.rstrip(i[-1]) for i in mep]
cclstrip = [i.rstrip(i[-1]) for i in ccl]

In [8]:
intersectindex = [i for i in cclstrip if i in mepstrip]
uniqueindex = [i for i in mepstrip if i not in cclstrip]

# dudas respecto de cual revisa en cual lista

In [9]:
iolbonosdf.drop(exclude, inplace=True)

# modifico el dataframe

In [10]:
tocalc = iolbonosdf.loc[mep + ccl + uniqueindex + intersectindex, ["Precio_Compra", "Precio_Venta"]]
tocalc.sort_index(inplace = True)
tocalc.loc[["CO26", "CO26D", "AE38C"]]

# crea un nuevo dataframe reducido, esto no es enteramente necesario, solo para visualizacion

,Precio_Compra,Precio_Venta
Simbolo,,
CO26,15956.5,16100.0
CO26D,54.4,54.8
AE38C,0.0,0.0


In [11]:
for i in cclstrip:
    a = tccompra(iolbonosdf["Precio_Venta"][i], iolbonosdf["Precio_Compra"][i + "C"], "black")
    b = tcventa(iolbonosdf["Precio_Compra"][i], iolbonosdf["Precio_Venta"][i + "C"], "black")
    print(i, b, a)

AE38 inf inf
AL29 inf inf
AL30 107.55 inf
AL35 inf inf
AL41 inf inf
GD29 inf inf
GD30 296.64 306.29
GD35 294.93 309.08
GD38 inf inf
GD41 inf inf
GD46 inf inf


C:\Users\usuario\AppData\Local\Temp\ipykernel_7212\2141870018.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  return round((ars * (1 + comissions[fee]))/(usd * (1 - comissions[fee])), 2)
C:\Users\usuario\AppData\Local\Temp\ipykernel_7212\2141870018.py:2: RuntimeWarning: divide by zero encountered in double_scalars
  return round((ars * (1 - comissions[fee]))/(usd * (1 + comissions[fee])), 2)


In [12]:
for i in mepstrip:
    a = tccompra(iolbonosdf["Precio_Venta"][i], iolbonosdf["Precio_Compra"][i + "D"], fee="black")
    b = tcventa(iolbonosdf["Precio_Compra"][i], iolbonosdf["Precio_Venta"][i + "D"], fee="black")
    print(i, b, a)

AE38 290.02 294.58
AL29 285.96 295.66
AL30 291.16 292.68
AL35 289.58 296.38
AL41 284.49 296.1
CO26 290.6 296.55
GD29 262.52 297.36
GD30 290.92 292.15
GD35 281.56 297.6
GD38 277.63 298.71
GD41 289.22 306.05
GD46 279.91 310.25


In [13]:
ticker = {"Simbolo": mepstrip}
compra = {}
venta = {}

calccompra = []
calcventa = []

for key in mepstrip:
    b = tcventa(iolbonosdf["Precio_Compra"][key], iolbonosdf["Precio_Venta"][key + "D"], fee="gold")
    calcventa.append(b)
    venta.update({"Compra": calcventa})
    
    a = tccompra(iolbonosdf["Precio_Venta"][key], iolbonosdf["Precio_Compra"][key + "D"], fee="gold")
    calccompra.append(a)
    compra.update({"Venta": calccompra})
    
# el For Loop inlcuye el argument fee en ambas variables - habría q reducirlo a una sola

In [14]:
finaldict = {}

finaldict.update(ticker)
finaldict.update(venta)
finaldict.update(compra)

finaldf = pandas.DataFrame(finaldict)

finaldf.set_index("Simbolo", inplace=True)

finaldf

# falta pasar los index de la lista mepstrip

,Compra,Venta
Simbolo,,
AE38,287.71,296.95
AL29,283.68,298.03
AL30,288.84,295.03
AL35,287.27,298.76
AL41,282.22,298.48
CO26,288.28,298.93
GD29,260.43,299.74
GD30,288.60,294.50
GD35,279.32,299.99


In [15]:
finaldf.describe()

,Compra,Venta
count,12.000000,12.000000
mean,282.195833,300.230833
std,8.263928,5.295927
min,260.430000,294.500000
25%,278.910000,297.760000
50%,285.295000,298.845000
75%,287.852500,300.270000
max,288.840000,312.740000


In [16]:
finaldf.sort_values(by="Compra")

,Compra,Venta
Simbolo,,
GD29,260.43,299.74
GD38,275.41,301.11
GD46,277.68,312.74
GD35,279.32,299.99
AL41,282.22,298.48
AL29,283.68,298.03
GD41,286.91,308.51
AL35,287.27,298.76
AE38,287.71,296.95


In [17]:
finaldf.sort_values(by="Venta")

,Compra,Venta
Simbolo,,
GD30,288.60,294.50
AL30,288.84,295.03
AE38,287.71,296.95
AL29,283.68,298.03
AL41,282.22,298.48
AL35,287.27,298.76
CO26,288.28,298.93
GD29,260.43,299.74
GD35,279.32,299.99
